## Finetuning YOLOv11 with Buffalo and Camel labels.
For installation and setup, view the [README](./README.md) before running this.

As mentioned in the README, we will be using Roboflow for labelling our dataset.

The Roboflow project is [available here](https://app.roboflow.com/stuff-avvl2/yolo-fine-y444l).

## Download Annotated Dataset
We will download the versioned dataset from Roboflow.

## Training the Model

### Setup the logging

Ultralytics support logging to wandb, comet.ml and tensorboard, out of the box. Here we only enable wandb.

You need to create an account at [wandb](https://wandb.ai) and get the API key from https://wandb.ai/authorize.

In [ ]:
from ultralytics import settings

settings.update({"wandb": True,
                 "tensorboard": False})

### Training

We specify the path to data.yaml file, and train with a batch size of 15, and we also save the checkpoint at each epoch (save_period=1). We assume here you are connected to a GPU, hence we can specify the device to use as `device=0` to select the first GPU.  We specify the project name as balloon, this will create a folder called `balloon` to store the weights and various training artifacts such as F1, PR curves, confusion matrics, training results (loss, mAP, etc).

For a complete listing of train settings, you can see [here](https://docs.ultralytics.com/modes/train/#train-settings).

You can also specify the type of data [augmentation](https://docs.ultralytics.com/modes/train/#augmentation-settings-and-hyperparameters)  you want as part of the train pipeline.

You can monitor your training progress at wandb (the link is given in the train output below)